In [1]:
import numpy as np
import time
import math
from datetime import datetime
from datetime import timedelta
import pandas as pd
import requests
import json
import pymongo
import itertools

In [2]:
from IPython.display import clear_output

In [3]:
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

In [4]:
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords

In [5]:
def create_ListofDict_InsertOne(df):
    info = {
            "Category": 'Entertain',
            "URL": df.link,
            "headline": df.title,
            "newsdate":df.article_date,
            "totalView": int(df['view']),
            "totalShare": int(df.share),
            "text": df.content_text,
            "hl_sentiment_score": df.hl_sentiment_score,
            "hl_polarity": df.hl_polarity,
            "hl_neg_kw": df.hl_neg_kw,
            "hl_pos_kw": df.hl_pos_kw,
            "t_sentiment_score": df.t_sentiment_score,
            "t_polarity": df.t_polarity,
            "t_neg_kw": df.t_neg_kw,
            "t_pos_kw": df.t_pos_kw,
            "dateDiff": float(df.dateDiff), 
            "avg_share_per_day": float(df.avg_share_per_day),
            "avg_view_per_day": float(df.avg_view_per_day)
    }
    return info

In [6]:
postNum = 30
postNumSleep = math.ceil(60/postNum)

url = "https://api.aiforthai.in.th/ssense"
headers = {
    'Apikey': "ymYbYiJ89ha609Nnv7wuV1CWJFLyCEYo"
    }

In [7]:
def plotSentiment_Dist(op, polarity,topic):
    t_cat_ratio = op.groupby([polarity,'Category']).agg({topic: lambda x: len(x)}).reset_index('Category')
    d_ax1 = t_cat_ratio[t_cat_ratio.Category=='Crime'][topic]
    d_ax2 = t_cat_ratio[t_cat_ratio.Category=='Politic'][topic]
    d_ax3 = t_cat_ratio[t_cat_ratio.Category=='Entertain'][topic]

    bar_label1,bar_label2,bar_label3 = d_ax1.index,d_ax2.index,d_ax3.index

    fig, ax = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
    ax[0].grid(color='k', linestyle='-', linewidth=0.2)
    ax[0].bar(bar_label1, d_ax1)
    ax[0].set_xticklabels(d_ax1.index)
    ax[0].set_title(f'Sentiment distribution : {topic} - Crime')
    ax[0].set_xlabel('Sentiment')
    ax[0].set_ylabel('count_news')

    ax[1].grid(color='k', linestyle='-', linewidth=0.2)
    ax[1].bar(bar_label2, d_ax2)
    ax[1].set_xticklabels(d_ax2.index)
    ax[1].set_title(f'Sentiment distribution : {topic} - Politic')
    ax[1].set_xlabel('Sentiment')
    ax[1].set_ylabel('count_news')

    ax[2].grid(color='k', linestyle='-', linewidth=0.2)
    ax[2].bar(bar_label3, d_ax3)
    ax[2].set_xticklabels(d_ax3.index)
    ax[2].set_title(f'Sentiment distribution : {topic} - Entertain')
    ax[2].set_xlabel('Sentiment')
    ax[2].set_ylabel('count_news')

    plt.show()

### Mark sentiment

In [8]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_k_urlList = mydb["SENTIMENT_KAPOOK"]
i_k_scrape = mydb["SENTIMENT_KAPOOK_API"]

In [9]:
cursor_kp = q_k_urlList.find()
cursor_i_kp = i_k_scrape.find()

df_kp = pd.DataFrame(cursor_kp)

In [10]:
df_kp = df_kp[df_kp.taxonomy.isin(['women/ดารา','women','movie/ข่าววงการหนัง','wedding/งานแต่งดารา','movie/Gossip Star(ซุบซิบดารา)'
                                                              ,'baby/แม่-ลูกดารา','home/บ้านดารา','drama/ข่าวละคร','musicstation/ข่าววงการเพลง','men/HotGirls'
                                                              ,'women/รูปภาพ','men/เกาะติดคนดังชาย','drama','pet/สัตว์เลี้ยงดารา','drama/เรื่องย่อละคร'
                                                              ,'women/ผู้หญิงน่ารู้','women/เมนูอาหาร'])]

In [11]:
current_day = datetime.strptime(str('2020-10-25 23:59:00'), '%Y-%m-%d %H:%M:%S')
df_kp['dateDiff']  = df_kp['article_date'].apply(lambda x : current_day - x )
df_kp['dateDiff']  = df_kp['dateDiff'].apply(lambda x : x.days)
df_kp['avg_share_per_day']  = df_kp['share'] / df_kp['dateDiff']
df_kp['avg_view_per_day']  = df_kp['view'] / df_kp['dateDiff']

In [12]:
df_kp.describe()

,view,share,image_count,share/view,dateDiff,avg_share_per_day,avg_view_per_day
count,4857.000000,4857.000000,4857.000000,4857.000000,4857.000000,4857.000000,4857.000000
mean,24533.561458,624.385217,4.436895,0.026940,149.044678,7.603696,370.602752
std,22935.930800,3141.944345,2.612340,0.092214,86.396944,37.121299,839.168889
min,378.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.678082
25%,10832.000000,85.000000,3.000000,0.004611,73.000000,0.664234,69.878378
50%,17769.000000,174.000000,4.000000,0.010531,145.000000,1.512438,144.421569
75%,30205.000000,411.000000,5.000000,0.023529,227.000000,3.780488,320.980198
max,351844.000000,100735.000000,49.000000,3.772442,299.000000,1024.777778,16818.666667


In [13]:
df_plotViz = df_kp.copy()

In [14]:
df_plotViz.drop('_id', axis=1,inplace=True)

In [15]:
mind = datetime.strptime(str('2020-07-01 00:00:00'), '%Y-%m-%d %H:%M:%S')
maxd = datetime.strptime(str('2020-10-24 23:59:00'), '%Y-%m-%d %H:%M:%S')
# เอาเฉพาะ avg_share_per_day สูงกว่า mean หมวดข่าวนั้น
df_plotViz = df_plotViz[(df_plotViz.avg_share_per_day>=7.603696)&(df_plotViz.article_date>=mind)&(df_plotViz.article_date<=maxd)]
df_plotViz.shape

(457, 16)

In [16]:
df_plotViz.insert(13,'hl_sentiment_score',value=None)
df_plotViz.insert(14,'hl_polarity',value=None)
df_plotViz.insert(15,'hl_neg_kw',value=None)
df_plotViz.insert(16,'hl_pos_kw',value=None)
df_plotViz.insert(17,'t_sentiment_score',value=None)
df_plotViz.insert(18,'t_polarity',value=None)
df_plotViz.insert(19,'t_neg_kw',value=None)
df_plotViz.insert(20,'t_pos_kw',value=None)

In [17]:
sentiment_db = mydb["SENTIMENT_KAPOOK_API"]
sentiment_db.delete_many({})

In [18]:
error_elem_h,error_elem_t = [],[]
apilist_header = []

for i,e in enumerate(df_plotViz.link,0):
    print('Round:',i)
    df_process = df_plotViz.iloc[i].copy().fillna('')
    h_data,t_data = {'text':df_process.title.replace('%', '').encode('utf-8')},{'text':df_process.content_text.replace('%', '').encode('utf-8')}
    try:
        h_response = requests.post(url, data=h_data, headers=headers)
        h_score = h_response.json()['sentiment']['score']
        h_polarity = h_response.json()['sentiment']['polarity']
        h_neg_kw = h_response.json()['preprocess']['neg']
        h_pos_kw = h_response.json()['preprocess']['pos']
        df_process.hl_sentiment_score,df_process.hl_polarity,df_process.hl_neg_kw,df_process.hl_pos_kw = h_score, h_polarity, h_neg_kw, h_pos_kw
    except:
        error_elem_h.append(i)
    ####
    try:
        t_response = requests.post(url, data=t_data, headers=headers)
        t_score = t_response.json()['sentiment']['score']
        t_polarity = t_response.json()['sentiment']['polarity']
        t_neg_kw = t_response.json()['preprocess']['neg']
        t_pos_kw = t_response.json()['preprocess']['pos']
        df_process.t_sentiment_score,df_process.t_polarity,df_process.t_neg_kw,df_process.t_pos_kw = t_score, t_polarity, t_neg_kw, t_pos_kw
    except:
        error_elem_t.append(i)
    process_data = create_ListofDict_InsertOne(df_process)
    print(process_data)
    sentiment_db.insert_one(process_data)
    time.sleep(postNumSleep)
    clear_output(wait=True)

Round: 456
{'Category': 'Entertain', 'URL': 'https://women.kapook.com/view232781.html', 'headline': 'หนูเล็ก ก่อนบ่าย ร้องให้ลูกเข้าโรงเรียนวันแรก เครียด น้องวิน แพ้เพิ่มทุกวัน', 'newsdate': Timestamp('2020-10-24 04:16:31'), 'totalView': 9735, 'totalShare': 452, 'text': 'หนูเล็ก ก่อนบ่าย เกาะรั้วร้องไห้ส่ง น้องวินเข้าโรงเรียนวันแรก พร้อมอัปเดตอาการแพ้ของลูก หนักขึ้นทุกวัน แต่ยังโชคดีที่ทานปลาสลิดได้ หวังอนาคตลูกจะหายดีเป็นปกติ ปกติก็เป็นคนเซนซิทีฟอยู่แล้วสําหรับคุณแม่สายฮา หนูเล็ก ก่อนบ่าย ยิ่งต้องส่งลูกเข้าโรงเรียนวันแรก ตัวเองถึงกับต้องเกาะรั้วร้องไห้ พร้อมอัปเดตอาการแพ้ของน้องวิน ลูกชาย ที่ทําเอาหัวอกคนเป็นแม่ถึงกับปวดใจ เมื่อลูกอยากกินอะไร แล้วต้องบอกว่า กินไม่ได้เพราะลูกแพ้หนัก ล่าสุด (23 ตุลาคม 2563) วันบันเทิง เผยบทสัมภาษณ์ หนูเล็ก ที่ได้มาอัพเดตพัฒนาการของน้องวิน ลูกชายวัย 1 ขวบ 7 เดือน ว่าตอนนี้พาเข้าโรงเรียนเรียบร้อยแล้ว และเมื่อวันที่ 20 ตุลาคม ที่ผ่านมา ก็เป็นวันแรกที่ลูกชายอยู่ที่โรงเรียนคนเดียว โดยที่ไม่มีพ่อแม่อยู่ด้วย เพราะก่อนหน้านี้พ่อกับแม่เข้าไปอยู่ในห้องเรียนกับลูก

In [19]:
# Element รันมือเพื่อตามเก็บ
print('Error in h:',error_elem_h,'Error in t',error_elem_t)

Error in h: [] Error in t []


### 